In [1]:
from pyspark.sql import SparkSession

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
saprk = SparkSession.builder.appName("market").getOrCreate()

In [3]:
df = spark.read.csv("customer_churn.csv",inferSchema=True,header=True)

In [5]:
for x in df.head(7):
    print(x)
    print("\n")

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)


Row(Names='Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location='6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company='Wilson PLC', Churn=1)


Row(Names='Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date=datetime.datetime(2016, 6, 29, 6, 20, 7), Location='1331 Keith Court Alyssahaven, DE 90114', Company='Miller, Johnson and Wallace', Churn=1)


Row(Names='Phillip White', Age=42.0, Total_Purchase=8010.76, Account_Manager=0, Years=6.71, Num_Sites=10.0, Onboard_date=datetime.datetime(2014, 4, 22, 12, 43, 12), Location='13120 Daniel Mount Angelabury, WY 30645-4695',

In [6]:
from pyspark.sql.functions import corr

In [15]:
df.select(corr("Churn", "Num_Sites")).show()

+----------------------+
|corr(Churn, Num_Sites)|
+----------------------+
|    0.5253980778988574|
+----------------------+



In [17]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [16]:
from pyspark.ml.feature import VectorAssembler 

In [22]:
vr = VectorAssembler(outputCol="feature",inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'])

In [26]:
final = vr.transform(df)
final.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|             feature|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

In [28]:
final = final.select("feature","Churn")

In [29]:
train,test = final.randomSplit([0.7,0.3])

In [30]:
from pyspark.ml.classification import LogisticRegression

In [31]:
lr = LogisticRegression(featuresCol='feature', labelCol='Churn')

In [32]:
lr = lr.fit(train)

In [36]:
ev = lr.evaluate(test)

In [38]:
ev.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|             feature|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.89190325667922...|[0.99254881604717...|       0.0|
|[25.0,9672.03,0.0...|    0|[4.72175971952768...|[0.99117899841009...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.70099174355070...|[0.66840761779228...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.26530204873259...|[0.98614697860257...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.83260951989280...|[0.99209724359686...|       0.0|
|[29.0,10203.18,1....|    0|[3.92340843896060...|[0.98060983041718...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.49443352613117...|[0.97052897058095...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.45529992700508...|[0.96938880333129...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.26324058534184...|[0.99809856009535...|       0.0|
|[30.0,10183.98,

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [46]:
bce = BinaryClassificationEvaluator(labelCol='Churn',rawPredictionCol='prediction')

In [47]:
bce = bce.evaluate(ev.predictions)

In [48]:
bce

0.7335750384530872

## Importing TestSet New-dataset

In [49]:
flr = LogisticRegression(featuresCol='feature', labelCol='Churn')

In [50]:
flr = flr.fit(final)

In [52]:
finalt =spark.read.csv("new_customers.csv",inferSchema=True,header=True)

In [53]:
finalt = vr.transform(finalt)

In [59]:
lrt= flr.transform(finalt)

In [60]:
lrt.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'feature',
 'rawPrediction',
 'probability',
 'prediction']

In [62]:
lrt.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

